In [4]:
from web3 import Web3, HTTPProvider
import random 

random.seed(10)
contract_manager = Web3.to_checksum_address(0xf421B69A6c4c3D911Bb3A4A8CFF29262618fC4B7)

In [5]:
# Connect to the chain

from web3.middleware import geth_poa_middleware

w3 = Web3(HTTPProvider('http://localhost:7545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.default_account = contract_manager

In [6]:
# New a DecentralElect Contract in the chain

from DecentralElect import PartitionDecentralElectContract
import json

import os

decentral_elect = PartitionDecentralElectContract(w3)
tx_rcpt = decentral_elect.deploy(contract_manager)
print('Contract Address:', tx_rcpt.contractAddress)


Contract Address: 0xB47F24A2cADCCFA56A9df8625E199c3E67A78d7b


In [7]:
# Initilize the vote
# New the vote, and add the subject and candidates to it.

election_list =[
    {
        'id': 0, 
        'subject': 'What fruit do you like?',
        'candidates': [
            {'name': 'Apple',
             'description': 'Red'}, 
            {'name': 'Grape',
             'description': 'Purple'}, 
            {'name': 'Orange',
             'description': 'Orange'}, 
            {'name': 'Guava',
             'description': 'Green'}, 
            {'name': 'Banana',
             'description': 'Yellow'}
            ]
    },
    {
        'id': 1, 
        'subject': 'What fruit do you like?',
        'candidates': [
            {'name': 'Apple',
             'description': 'Red'}, 
            {'name': 'Grape',
             'description': 'Purple'}, 
            {'name': 'Orange',
             'description': 'Orange'}, 
            {'name': 'Guava',
             'description': 'Green'}, 
            {'name': 'Banana',
             'description': 'Yellow'}
            ]
    },
    {
        'id': 2, 
        'subject': 'What fruit do you like?',
        'candidates': [
            {'name': 'Apple',
             'description': 'Red'}, 
            {'name': 'Grape',
             'description': 'Purple'}, 
            {'name': 'Orange',
             'description': 'Orange'}, 
            {'name': 'Guava',
             'description': 'Green'}, 
            {'name': 'Banana',
             'description': 'Yellow'}
            ]
    },
    {
        'id': 3, 
        'subject': 'What fruit do you like?',
        'candidates': [
            {'name': 'Apple',
             'description': 'Red'}, 
            {'name': 'Grape',
             'description': 'Purple'}, 
            {'name': 'Orange',
             'description': 'Orange'}, 
            {'name': 'Guava',
             'description': 'Green'}, 
            {'name': 'Banana',
             'description': 'Yellow'}
            ]
    },
    {
        'id': 4, 
        'subject': 'What fruit do you like?',
        'candidates': [
            {'name': 'Apple',
             'description': 'Red'}, 
            {'name': 'Grape',
             'description': 'Purple'}, 
            {'name': 'Orange',
             'description': 'Orange'}, 
            {'name': 'Guava',
             'description': 'Green'}, 
            {'name': 'Banana',
             'description': 'Yellow'}
            ]
    },
]

for election in election_list:
    print('Status:', decentral_elect.get_status(election['id']))
    tx_rcpt = decentral_elect.create_election(identifier = election['id'], subject = election['subject'])
    print('Status:', decentral_elect.get_status(election['id']))

    for candidate in election['candidates']:
        decentral_elect.set_candidate(identifier = election['id'], name = candidate['name'], description = candidate['description'])

    tx_rcpt = decentral_elect.get_candidates(election['id'])
    print(tx_rcpt)


Status: Unopened
Status: Registration
[(0, 'Apple', 'Red', 0), (1, 'Grape', 'Purple', 0), (2, 'Orange', 'Orange', 0), (3, 'Guava', 'Green', 0), (4, 'Banana', 'Yellow', 0)]
Status: Unopened
Status: Registration
[(0, 'Apple', 'Red', 0), (1, 'Grape', 'Purple', 0), (2, 'Orange', 'Orange', 0), (3, 'Guava', 'Green', 0), (4, 'Banana', 'Yellow', 0)]
Status: Unopened
Status: Registration
[(0, 'Apple', 'Red', 0), (1, 'Grape', 'Purple', 0), (2, 'Orange', 'Orange', 0), (3, 'Guava', 'Green', 0), (4, 'Banana', 'Yellow', 0)]
Status: Unopened
Status: Registration
[(0, 'Apple', 'Red', 0), (1, 'Grape', 'Purple', 0), (2, 'Orange', 'Orange', 0), (3, 'Guava', 'Green', 0), (4, 'Banana', 'Yellow', 0)]
Status: Unopened
Status: Registration
[(0, 'Apple', 'Red', 0), (1, 'Grape', 'Purple', 0), (2, 'Orange', 'Orange', 0), (3, 'Guava', 'Green', 0), (4, 'Banana', 'Yellow', 0)]


In [8]:
from DecentralElect import Voter

register_list = [
    {
        'id': 0,
        'registers' : [Voter() for _ in range(10)]
    },
    # {
    #     'id': 1,
    #     'registers' : [Voter() for _ in range(15000)]
    # },
    # {
    #     'id': 2,
    #     'registers' : [Voter() for _ in range(20000)]
    # },
    # {
    #     'id': 3,
    #     'registers' : [Voter() for _ in range(25000)]
    # },
    # {
    #     'id': 4,
    #     'registers' : [Voter() for _ in range(30000)]
    # },
]

for election in register_list:
    count = 0
    for register in election['registers']:
        decentral_elect.register_voter(election['id'], register.public_key[0].n, register.public_key[1].n)
        if not decentral_elect.check_if_registered(election['id'], register.public_key[0].n):
            print('Error', register)
        # tx_rcpt = decentral_elect.get_constituency()

In [9]:
for election in register_list:
    progress, length = decentral_elect.get_cluster_progress(election['id'])
    
    while(progress < length):
        decentral_elect.cluser_voters(election['id'])
        progress, length = decentral_elect.get_cluster_progress(election['id'])
        print('Progress:{}/{}'.format(progress, length))

Progress:10/10


In [12]:
from DecentralElect.uaosring import uaosring_sign
from os import urandom
from DecentralElect import hash_vote
from DecentralElect.utils import bytes_to_int
from py_ecc.bn128 import FQ
from DecentralElect.linkable_ring_signature import LinkableRingSignature
import random
import string

def generate_random_strings(num_strings:int, leading_numbers:list[int]):
    random_strings = []

    for _ in range(num_strings):
        leading_number = random.choice(leading_numbers)
        
        rest_of_string = ''.join(random.choices(string.ascii_letters + string.digits, k=20))
    
        random_string = str(leading_number) + '_' + rest_of_string
        random_strings.append(random_string)

    return random_strings

voting_list = register_list
voting_list[0]['vote'] = generate_random_strings(5, [0 ,1, 2, 3, 4])
# voting_list[1]['vote'] = generate_random_strings(15000, [0 ,1, 2, 3, 4])
# voting_list[2]['vote'] = generate_random_strings(20000, [0 ,1, 2, 3, 4])
# voting_list[3]['vote'] = generate_random_strings(25000, [0 ,1, 2, 3, 4])
# voting_list[4]['vote'] = generate_random_strings(30000, [0 ,1, 2, 3, 4])

def pkeys_type_transform(pkeys:list[FQ]) -> list:
    result = []
    for x, y in pkeys:
        result.append((FQ(x), FQ(y)))

    return result

def pack_vote_in_random32(vote: bytes) -> bytes:
    assert(len(vote) <= 32)
    return urandom(32 - len(vote)) + vote

for election in voting_list:
    decentral_elect.start_vote(election['id'])
    print('Status:', decentral_elect.get_status(election['id']))

    for ballot, voter in zip(election['vote'], election['registers']):
        constituency = decentral_elect.get_constituency(election['id'], voter.public_key[0].n)
        pkeys = decentral_elect.get_registered_pkeys(election['id'], constituency)
        pkeys = pkeys_type_transform(pkeys)

        message = hash_vote(ballot)
        signature = uaosring_sign(pkeys, (voter.public_key, voter.private_key), message)

        linkable_ring_signature = LinkableRingSignature(
            pkeys = pkeys, 
            tag = signature[1], 
            ring = signature[2], 
            seed = signature[3], 
            message = message
            )
        decentral_elect.vote(election['id'], constituency, linkable_ring_signature)

        if (decentral_elect.check_if_voted(election['id'], linkable_ring_signature.tag[0].n)):
            print('Voted')
        else:
            print("UnVoted")


Status: Vote
Voted
Voted
Voted
Voted
Voted


In [43]:
message = hash_vote('2_apple')
v = voting_list[0]['registers'][3]
identifier = 0
constituency = decentral_elect.get_constituency(identifier, v.public_key[0].n)
pkeys = decentral_elect.get_registered_pkeys(identifier, constituency)
pkeys = pkeys_type_transform(pkeys)

signature = uaosring_sign(pkeys, (v.public_key, v.private_key), message)
# signature[2][0] += 3


linkable_ring_signature = LinkableRingSignature(
    pkeys = pkeys, 
    tag = signature[1], 
    ring = signature[2], 
    seed = signature[3], 
    message = message
    )

linkable_ring_signature.tag[0].n = 8057056557119254816153517554109639926615808794153727907915446913944040765669
linkable_ring_signature.tag[1].n = 5681741858004897999563955774191376641471322585285955070118289987066955470434

decentral_elect.vote(identifier, constituency, linkable_ring_signature)

if (decentral_elect.check_if_voted(identifier, linkable_ring_signature.tag[0].n)):
    print('Voted')
else:
    print("UnVoted")


UnVoted


In [41]:
from DecentralElect.altbn128 import *
from DecentralElect.utils import *


a = Voter()

def pkeys_hash_calculator(pkeys):
	assert len(pkeys) > 0
	hash_acc = hashs(pkeys[0][0].n)
	for pk in pkeys[1:len(pkeys)]:
		hash_acc = hashs(hash_acc, pk[0].n)
	return hash_acc

L = hashtopoint(pkeys_hash_calculator(pkeys))
T = multiply(L, a.private_key)

print(T)

(8057056557119254816153517554109639926615808794153727907915446913944040765669, 5681741858004897999563955774191376641471322585285955070118289987066955470434)


In [9]:
# verify ballot
from DecentralElect.utils import int_to_big_endian

result_list = voting_list

for election in result_list:
    decentral_elect.stop_vote(election['id'])

    # ballots = decentral_elect.get_ballots(election['id'])

    for ballot in election['vote']:
        decentral_elect.verify_ballot(election['id'], ballot)

In [10]:
# Show result

for election in result_list:
    candidates = decentral_elect.get_candidates(election['id'])
    print(election['id'])
    for candidate in candidates:
        print(candidate)

0
(0, 'Apple', 'Red', 2)
(1, 'Grape', 'Purple', 2)
(2, 'Orange', 'Orange', 3)
(3, 'Guava', 'Green', 3)
(4, 'Banana', 'Yellow', 0)


In [11]:
print(decentral_elect.voter_gas_spend + decentral_elect.manager_gas_spend)

14710965


In [3]:
from DecentralElect import Voter

print(Voter())
print(Voter())

Private: 4668143976994134550021904368266290461569141490433399027785177489270673113786
Public: (6976498423903177657941087729755215471712710429741876987755298396731327586177, 19962525593850070673721010759498457750839098993983563732690242872569236366549)
Private: 14075061314077086207845970169164224906288783062399247027778967797315681463048
Public: (13800384787206396823867946942174405058636793170637014147152501483492582420055, 11546564977006419542926248029165476053728284261558833467247652695425759726232)
